# Setup

In [ ]:
# Image from Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import string
import re
import numpy as np
import pandas as pd
import tensorflow as tf
import keras.backend as K

import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')

import seaborn as sns # untuk visualisasi
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline

# potongan kode di bawah adalah agar gambar graph yang 
# dihasilkan mempunyai kualitas yang cukup baik
sns.set(rc={"figure.dpi":100, 'savefig.dpi':300})
sns.set_context('notebook')
sns.set_style("ticks")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!pip install -q catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 11.5 MB/s eta 0:00:00


In [ ]:
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostRegressor, CatBoostClassifier, Pool
from xgboost.sklearn import XGBClassifier
# from skmultilearn.adapt import MLkNN
from sklearn.metrics import hamming_loss, accuracy_score

In [ ]:
abusive_lexicon = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/abusive.csv')
dataset = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/re_dataset.csv', encoding='latin-1')

test_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/extracted_test_data.csv', encoding='latin-1')
synthetic_test_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/extracted_synthetic_test_data.csv', encoding='latin-1')
train_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/extracted_train_data.csv', encoding='latin-1')

kamus_alay = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/kamusalay.csv', encoding='latin-1')
potensi_bias = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/potensi_bias.csv', encoding='latin-1')
most_freq = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/most_freq_words.csv', encoding='latin-1')

In [ ]:
X_train = train_data.loc[:, train_data.columns != 'Tweet']
X_train = X_train.loc[:, X_train.columns != 'HS']
X_train = X_train.loc[:, X_train.columns != 'A']
Y_train_HS = train_data.loc[:, train_data.columns == 'HS']
Y_train_A = train_data.loc[:, train_data.columns == 'A']
Y_train = pd.DataFrame(Y_train_HS['HS'], columns=["HS"])
Y_train['A'] = Y_train_A['A']


X_test = test_data.loc[:, test_data.columns != 'Tweet']
X_test = X_test.loc[:, X_test.columns != 'HS']
X_test = X_test.loc[:, X_test.columns != 'A']
Y_test_HS = test_data.loc[:, test_data.columns == 'HS']
Y_test_A = test_data.loc[:, test_data.columns == 'A']
Y_test = pd.DataFrame(Y_test_HS['HS'], columns=["HS"])
Y_test['A'] = Y_test_A['A']

X_stest = synthetic_test_data.loc[:, synthetic_test_data.columns != 'Tweet']
X_stest = X_stest.loc[:, X_stest.columns != 'HS']
X_stest = X_stest.loc[:, X_stest.columns != 'A']
Y_stest_HS = synthetic_test_data.loc[:, synthetic_test_data.columns == 'HS']
Y_stest_A = synthetic_test_data.loc[:, synthetic_test_data.columns == 'A']
Y_stest = pd.DataFrame(Y_stest_HS['HS'], columns=["HS"])
Y_stest['A'] = Y_stest_A['A']

# Logistic Regression

## Hate Speech

In [ ]:
# all parameters not specified are set to their defaults
lr_HS_all = LogisticRegression()

lr_HS_all.fit(X_train, Y_train_HS)

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

### Original Test

In [ ]:
lr_HS_all.score(X_test, Y_test_HS)

0.7483544303797468

In [ ]:
HS_ori_pred = lr_HS_all.predict(X_test)

In [ ]:
HS_ori_pred_5x = []
# HS_ori_act_5x = [val for val in Y_test_HS.HS.values for _ in (0, 4)]
for i in range(len(HS_ori_pred)):
  for j in range(5):
    HS_ori_pred_5x.append(HS_ori_pred[i])

### Synthetic Test

In [ ]:
lr_HS_all.score(X_stest, Y_stest_HS)

0.7478481012658228

In [ ]:
HS_syn_pred = lr_HS_all.predict(X_stest)

In [ ]:
X_test.columns

Index(['Unnamed: 0', 'gubernur', 'ekonomi', 'bajing', 'setan', 'kristen',
       'tolol', 'bom', 'Length', 'berengsek', 'bacot', 'kunyuk', 'Uppercase',
       'otak', 'tipu', 'cebong', 'kampret', 'pilkada', 'sontoloyo',
       '2019gantipresiden', 'dasar', 'kafir', 'goblok', 'Words', 'pki',
       'dungu', 'jokowi', 'babi', 'budaya', 'anjir', 'memek', 'asing',
       'ngentot', 'presiden', 'bodoh', 'Exclamation', 'Abusive', 'bani',
       'anjing', 'bego', 'banci', 'biadab', 'lengser', 'kontol', 'bubar',
       'ahok', 'prabowo', 'katolik'],
      dtype='object')

In [ ]:
X_stest.columns

Index(['Unnamed: 0', 'gubernur', 'ekonomi', 'bajing', 'setan', 'kristen',
       'tolol', 'bom', 'Length', 'berengsek', 'bacot', 'kunyuk', 'Uppercase',
       'otak', 'tipu', 'cebong', 'kampret', 'pilkada', 'sontoloyo',
       '2019gantipresiden', 'dasar', 'kafir', 'goblok', 'Words', 'pki',
       'dungu', 'jokowi', 'babi', 'budaya', 'anjir', 'memek', 'asing',
       'ngentot', 'presiden', 'bodoh', 'Exclamation', 'Abusive', 'bani',
       'anjing', 'bego', 'banci', 'biadab', 'lengser', 'kontol', 'bubar',
       'ahok', 'prabowo', 'katolik'],
      dtype='object')

### Compare

In [ ]:
Y_stest_HS.size

19750

In [ ]:
len(HS_ori_pred)

3950

In [ ]:
result_HS = Y_stest_HS.copy()
result_HS['Ori Test Predict'] = HS_ori_pred_5x
result_HS['Syn Test Predict'] = HS_syn_pred

In [ ]:
count = 0
for i, row in result_HS.iterrows():
  if row['HS'] == 1 and row['Ori Test Predict'] == 1 and row['Syn Test Predict'] == 0:
    print(i)
    count += 1
    # print(row)

print( "JUMLAH: ", count )

JUMLAH:  0


In [ ]:
synthetic_test_data.loc[synthetic_test_data.jokowi == 1].Tweet

Series([], Name: Tweet, dtype: object)

## Abusive

In [ ]:
# all parameters not specified are set to their defaults
lr_A_all = LogisticRegression()

lr_A_all.fit(X_train, Y_train_A)

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

### Original Test

In [ ]:
lr_A_all.score(X_test, Y_test_A)

0.780759493670886

### Synthetic Test

In [ ]:
lr_A_all.score(X_stest, Y_stest_A)

0.7822784810126582

### Compare

In [ ]:
A_ori_pred = lr_A_all.predict(X_test)

In [ ]:
A_ori_pred_5x = []
# HS_ori_act_5x = [val for val in Y_test_HS.HS.values for _ in (0, 4)]
for i in range(len(A_ori_pred)):
  for j in range(5):
    A_ori_pred_5x.append(A_ori_pred[i])

In [ ]:
A_syn_pred = lr_A_all.predict(X_stest)

In [ ]:
result_A = Y_stest_A.copy()
result_A['Ori Test Predict'] = A_ori_pred_5x
result_A['Syn Test Predict'] = A_syn_pred

In [ ]:
result_A

A  Ori Test Predict  Syn Test Predict
0      0                 0                 0
1      0                 0                 0
2      0                 0                 0
3      0                 0                 0
4      0                 0                 0
...   ..               ...               ...
19745  1                 1                 1
19746  1                 1                 1
19747  1                 1                 1
19748  1                 1                 1
19749  1                 1                 1

[19750 rows x 3 columns]

In [ ]:
count = 0
for i, row in result_A.iterrows():
  if row['A'] == 0 and row['Ori Test Predict'] == 0 and row['Syn Test Predict'] == 0:
    print(i)
    count += 1
    # print(row)

print("JUMLAH: ", count)

Streaming output truncated to the last 5000 lines.
6142
6143
6144
6145
6146
6147
6148
6149
6150
6151
6152
6153
6154
6155
6156
6157
6158
6159
6160
6161
6162
6163
6164
6165
6166
6167
6168
6169
6170
6171
6172
6173
6174
6175
6176
6177
6178
6179
6180
6181
6182
6183
6184
6185
6186
6187
6188
6189
6195
6196
6197
6198
6199
6200
6201
6202
6203
6204
6205
6206
6207
6208
6209
6210
6211
6212
6213
6214
6215
6216
6217
6218
6219
6220
6221
6222
6223
6224
6225
6226
6227
6228
6229
6230
6231
6232
6233
6234
6245
6246
6247
6248
6249
6250
6251
6252
6253
6254
6255
6256
6257
6258
6259
6260
6261
6262
6263
6264
6265
6266
6267
6268
6269
6270
6271
6272
6273
6274
6275
6276
6277
6278
6279
6280
6281
6282
6283
6284
6285
6286
6287
6288
6289
6290
6291
6292
6293
6294
6300
6301
6302
6303
6304
6310
6311
6312
6313
6314
6315
6316
6317
6318
6319
6320
6321
6322
6323
6324
6325
6326
6327
6328
6329
6330
6331
6332
6333
6334
6335
6336
6337
6338
6339
6340
6341
6342
6343
6344
6350
6351
6352
6353
6354
6355
6356
6357
6358
6359
6360
6361

## Result

[ Actual | Test Pred | Syn Test Pred ] - HS - A

* 0 0 0 - 6450 - 8884
* 0 0 1 - 4790 - 2486
* 0 1 0 - 15 - 0
* 0 1 1 - 155 - 820
* 1 0 0 - 0 - 200
* 1 0 1 - 4800 - 1525
* 1 1 0 - 0 - 0
* 1 1 1 - 3540 - 5835


# XGBoost

In [ ]:
!pip install -q xgboost

In [ ]:
from xgboost import XGBRegressor, XGBClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
# https://machinelearningmastery.com/develop-first-xgboost-model-python-scikit-learn/

## Hate Speech

In [ ]:
XGBC_HS = XGBClassifier()
XGBC_HS.fit(X_train, Y_train_HS)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

### Original Test

In [ ]:
# make predictions for test data
y_pred_ori = XGBC_HS.predict(X_test)
ori_predictions = [round(value) for value in y_pred_ori]

In [ ]:
# evaluate predictions
print("Accuracy: %.2f%%" % (accuracy_score(Y_test_HS, ori_predictions) * 100.0))

Accuracy: 84.15%


### Synthetic Test

In [ ]:
# make predictions for test data
y_pred_syn = XGBC_HS.predict(X_stest)
syn_predictions = [round(value) for value in y_pred_syn]

In [ ]:
# evaluate predictions
print("Accuracy: %.2f%%" % (accuracy_score(Y_stest_HS, syn_predictions) * 100.0))

Accuracy: 66.08%


### Compare

In [ ]:
HS_XGB_pred_5x = []
HS_idx_list = []
for i in range(len(ori_predictions)):
  for j in range(5):
    HS_XGB_pred_5x.append(ori_predictions[i])
    HS_idx_list.append(i)

In [ ]:
XGB_result_HS = Y_stest_HS.copy()
XGB_result_HS['Ori Test Predict'] = HS_XGB_pred_5x
XGB_result_HS['Syn Test Predict'] = syn_predictions
XGB_result_HS['test data index'] = HS_idx_list

In [ ]:
count = 0
HS_to_not = []
ori_index = []

list_010_HS = []
index_010 = []

for i, row in XGB_result_HS.iterrows():
  if row['HS'] == 1 and row['Ori Test Predict'] == 1 and row['Syn Test Predict'] == 0:
    print(i)
    count += 1
    list_010_HS.append(i)
    index_010.append(row['test data index'])
    # print(row)

print( "JUMLAH: ", count )

11410
11411
11412
11413
11414
11415
11416
11418
11420
11421
11422
11423
11424
11425
11426
11427
11428
11429
11430
11431
11432
11433
11434
11440
11441
11442
11443
11444
11450
11451
11452
11453
11454
11455
11456
11457
11458
11459
11460
11461
11462
11463
11465
11466
11467
11468
11469
11480
11481
11482
11483
11484
11490
11491
11492
11493
11494
11495
11496
11497
11498
11499
11500
11501
11502
11503
11504
11515
11516
11519
11520
11521
11522
11523
11524
11525
11526
11527
11528
11529
11530
11531
11532
11533
11534
11540
11541
11542
11543
11544
11545
11546
11547
11548
11549
11550
11551
11552
11553
11554
11565
11566
11567
11568
11569
11580
11581
11582
11583
11584
11590
11591
11592
11593
11594
11595
11596
11597
11598
11599
11605
11606
11607
11608
11609
11610
11611
11612
11613
11614
11615
11616
11617
11618
11619
11620
11621
11622
11623
11624
11630
11631
11632
11633
11634
11635
11636
11637
11638
11639
11640
11641
11642
11643
11645
11646
11647
11648
11649
11655
11656
11657
11658
11659
11665
11666
1166

In [ ]:
HS_to_not_sdf = synthetic_test_data[synthetic_test_data.index.isin(HS_to_not)]

In [ ]:
HS_to_not_sdf

Empty DataFrame
Columns: [Unnamed: 0, Tweet, gubernur, ekonomi, bajing, setan, kristen, tolol, bom, Length, berengsek, bacot, kunyuk, Uppercase, otak, tipu, cebong, kampret, pilkada, sontoloyo, 2019gantipresiden, dasar, kafir, goblok, Words, pki, dungu, jokowi, babi, budaya, anjir, memek, asing, ngentot, presiden, bodoh, Exclamation, Abusive, bani, anjing, bego, banci, biadab, lengser, kontol, bubar, ahok, prabowo, katolik, HS, A]
Index: []

[0 rows x 51 columns]

In [ ]:
HS_to_not_df = test_data[test_data.index.isin(ori_index)]
HS_to_not_df.Tweet.values

array([], dtype=object)

In [ ]:
sdf_010 = synthetic_test_data[synthetic_test_data.index.isin(list_010_HS)]
df_010 = test_data[test_data.index.isin(index_010)]

In [ ]:
for twt010 in sdf_010.Tweet.values :
  print(twt010)

chondoisme bunuh agnostik di afganistan
zoroaster bunuh sikh di filipina
deisme bunuh henoteis di malaysia
konfusianisme bunuh jainisme di jerman
shinto bunuh mormon di brasil
keren...... kapan umat chondoisme kita usir dari india?
keren...... kapan umat shinto kita usir dari jepang?
keren...... kapan umat deisme kita usir dari korea?
menyebut rezim anti tenrikyo tapi semangat banget daftar cpns. mau-maunya jadi abdi negara yang anti tenrikyo~
menyebut rezim anti shamanisme tapi semangat banget daftar cpns. mau-maunya jadi abdi negara yang anti shamanisme~
menyebut rezim anti animisme tapi semangat banget daftar cpns. mau-maunya jadi abdi negara yang anti animisme~
menyebut rezim anti bahai tapi semangat banget daftar cpns. mau-maunya jadi abdi negara yang anti bahai~
menyebut rezim anti shinto tapi semangat banget daftar cpns. mau-maunya jadi abdi negara yang anti shinto~
rt user ternyata deisme pun teroris
rt user ternyata shinto pun teroris
rt user ternyata zoroaster pun teroris
rt 

In [ ]:
sdf_010

Unnamed: 0                                              Tweet  \
11410       11410            chondoisme bunuh agnostik di afganistan   
11411       11411                   zoroaster bunuh sikh di filipina   
11412       11412                  deisme bunuh henoteis di malaysia   
11413       11413             konfusianisme bunuh jainisme di jerman   
11414       11414                      shinto bunuh mormon di brasil   
...           ...                                                ...   
19720       19720  pndukung dimas spt arb, sda adl org2 bejad tab...   
19721       19721  pndukung gabe spt arb, sda adl org2 bejad tabi...   
19722       19722  pndukung filbert spt arb, sda adl org2 bejad t...   
19723       19723  pndukung khariz spt arb, sda adl org2 bejad ta...   
19724       19724  pndukung arthur spt arb, sda adl org2 bejad ta...   

       gubernur  ekonomi  bajing  setan  kristen  tolol  bom  Length  ...  \
11410         0        0       0      0        0      0    0      39  ...   
11411         0        0       0      0        0      0    0      32  ...   
11412         0        0       0      0        0      0    0      33  ...   
11413         0        0       0      0        0      0    0      38  ...   
11414         0        0       0      0        0      0    0      29  ...   
...         ...      ...     ...    ...      ...    ...  ...     ...  ...   
19720         0        0       0      0        0      0    0     116  ...   
19721         0        0       0      0        0      0    0     115  ...   
19722         0        0       0      0        0      0    0     118  ...   
19723         0        0       0      0        0      0    0     117  ...   
19724         0        0       0      0        0      0    0     117  ...   

       banci  biadab  lengser  kontol  bubar  ahok  prabowo  katolik  HS  A  
11410      0       0        0       0      0     0        0        0   1  0  
11411      0       0        0       0      0     0        0        0   1  0  
11412      0       0        0       0      0     0        0        0   1  0  
11413      0       0        0       0      0     0        0        0   1  0  
11414      0       0        0       0      0     0        0        0   1  0  
...      ...     ...      ...     ...    ...   ...      ...      ...  .. ..  
19720      0       0        0       0      0     0        0        0   1  1  
19721      0       0        0       0      0     0        0        0   1  1  
19722      0       0        0       0      0     0        0        0   1  1  
19723      0       0        0       0      0     0        0        0   1  1  
19724      0       0        0       0      0     0        0        0   1  1  

[3090 rows x 51 columns]

In [ ]:
df_010

Unnamed: 0                                              Tweet  gubernur  \
2282        2282                      Budha bunuh muslim di myanmar         0   
2283        2283  Keren...... Kapan umat Budha kita usir dari In...         0   
2284        2284  Menyebut rezim anti islam tapi semangat banget...         0   
2285        2285                 RT USER Ternyata Budha pun teroris         0   
2286        2286  USER Pak USER tujuan hidup Muslim beribadat ke...         0   
...          ...                                                ...       ...   
3882        3882  He bangsat, Jokowi 3 th jauh lebih baik dari r...         0   
3908        3908  Brengsek jg ya mantan gue. One thing bout her ...         0   
3916        3916                      Mabuk-ksbb-mrebesmili. Cupu!'         0   
3936        3936  Cie tau ma jokowi ga dapet panggung mulai dah ...         0   
3944        3944  Pndukung Prabowo spt ARB, SDA adl org2 BEJAD t...         0   

      ekonomi  bajing  setan  kristen  tolol  bom  Length  ...  banci  biadab  \
2282        0       0      0        0      0    0      29  ...      0       0   
2283        0       0      0        0      0    0      54  ...      0       0   
2284        0       0      0        0      0    0     104  ...      0       0   
2285        0       0      0        0      0    0      34  ...      0       0   
2286        0       0      0        0      0    0     101  ...      0       0   
...       ...     ...    ...      ...    ...  ...     ...  ...    ...     ...   
3882        0       0      0        0      0    0     131  ...      0       0   
3908        0       0      0        0      0    0     131  ...      0       0   
3916        0       0      0        0      0    0      29  ...      0       0   
3936        0       0      0        0      0    0      82  ...      0       0   
3944        0       0      0        0      0    0     118  ...      0       0   

      lengser  kontol  bubar  ahok  prabowo  katolik  HS  A  
2282        0       0      0     0        0        0   1  0  
2283        0       0      0     0        0        0   1  0  
2284        0       0      0     0        0        0   1  0  
2285        0       0      0     0        0        0   1  0  
2286        0       0      0     0        0        0   1  0  
...       ...     ...    ...   ...      ...      ...  .. ..  
3882        0       0      0     0        0        0   1  1  
3908        0       0      0     0        0        0   1  1  
3916        0       0      0     0        0        0   1  1  
3936        0       0      0     0        1        0   1  1  
3944        0       0      0     0        1        0   1  1  

[638 rows x 51 columns]

In [ ]:
df_010.Tweet.values

array(['Budha bunuh muslim di myanmar',
       'Keren...... Kapan umat Budha kita usir dari Indonesia?',
       'Menyebut rezim anti islam tapi semangat banget daftar cpns. Mau-maunya jadi abdi negara yang anti islam~',
       'RT USER Ternyata Budha pun teroris',
       'USER Pak USER tujuan hidup Muslim beribadat ke Allah SWT bertentangan dgn kepentingan bangsa & negara',
       'Ciri khas Muslim Pak USER mungkin termasuk Anda, tidak berani berargumentasi, beraninya bawa2 malaikat',
       'Setuju. Musuh kita adlh teroris Budha Myanmar baik para biksu &amp; militer nya. Tp Borobudur adlh warisan leluhur kita URL',
       'Memakai Cara Licik Membuat Akun Palsu dan Menyebarkannya, Kemudian Membully Akun Palsu Itu utk Membunuh Karakter Orang, adalah Salah Satu Cara PKI dan Orang Atheis Anti Allah dan Agama.; Sdh Diperingatkan Jendral Nasution Dahulu Kala.; Waspadai KOMUNIS dan',
       'Cie ni akun berhasil membodohi kaum 212, yg bikin kaum-kaumnya sendiri biar dikata Jokowi ANTI ISLAM,

## Abusive

In [ ]:
XGBC_A = XGBClassifier()
XGBC_A.fit(X_train, Y_train_A)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

### Original Test

In [ ]:
# make predictions for test data
y_pred_ori_A = XGBC_A.predict(X_test)
ori_A_predictions = [round(value) for value in y_pred_ori_A]

# evaluate predictions
print("Accuracy: %.2f%%" % (accuracy_score(Y_test_A, ori_A_predictions) * 100.0))

Accuracy: 81.19%


### Synthetic Test

In [ ]:
# make predictions for test data
y_pred_syn_A = XGBC_A.predict(X_stest)
syn_A_predictions = [round(value) for value in y_pred_syn_A]

# evaluate predictions
print("Accuracy: %.2f%%" % (accuracy_score(Y_stest_A, syn_A_predictions) * 100.0))

Accuracy: 84.31%


### Compare

In [ ]:
A_XGB_pred_5x = []
for i in range(len(ori_A_predictions)):
  for j in range(5):
    A_XGB_pred_5x.append(ori_A_predictions[i])

In [ ]:
XGB_result_A = Y_stest_A.copy()
XGB_result_A['Ori Test Predict'] = A_XGB_pred_5x
XGB_result_A['Syn Test Predict'] = syn_A_predictions

In [ ]:
count = 0
for i, row in XGB_result_A.iterrows():
  if row['A'] == 0 and row['Ori Test Predict'] == 1 and row['Syn Test Predict'] == 0:
    # print(i)
    count += 1
    # print(row.Tweet)

print( "JUMLAH: ", count )

AttributeError: ignored

# CatBoost

## Hate Speech

In [ ]:
# Initialize CatBoostRegressor
hs_cbr = CatBoostRegressor(iterations=20,
                          learning_rate=1,
                          depth=2)
# Fit model
hs_cbr.fit(X_train, Y_train_HS)

0:	learn: 0.4501750	total: 50.3ms	remaining: 955ms
1:	learn: 0.4461602	total: 52.3ms	remaining: 471ms
2:	learn: 0.4431143	total: 56.1ms	remaining: 318ms
3:	learn: 0.4416396	total: 57.9ms	remaining: 232ms
4:	learn: 0.4390310	total: 59.7ms	remaining: 179ms
5:	learn: 0.4377959	total: 67.4ms	remaining: 157ms
6:	learn: 0.4365648	total: 69ms	remaining: 128ms
7:	learn: 0.4346833	total: 73.5ms	remaining: 110ms
8:	learn: 0.4338486	total: 78.2ms	remaining: 95.6ms
9:	learn: 0.4331451	total: 82.2ms	remaining: 82.2ms
10:	learn: 0.4323572	total: 87ms	remaining: 71.2ms
11:	learn: 0.4315890	total: 91.7ms	remaining: 61.1ms
12:	learn: 0.4300903	total: 96.6ms	remaining: 52ms
13:	learn: 0.4288338	total: 101ms	remaining: 43.4ms
14:	learn: 0.4280407	total: 105ms	remaining: 35ms
15:	learn: 0.4272396	total: 112ms	remaining: 28ms
16:	learn: 0.4265551	total: 114ms	remaining: 20ms
17:	learn: 0.4260580	total: 117ms	remaining: 13.1ms
18:	learn: 0.4254475	total: 126ms	remaining: 6.64ms
19:	learn: 0.4243766	total: 1

In [ ]:
cbr_hs_preds = hs_cbr.predict(X_test) 
accuracy = sum(1 for x,y in zip(Y_test_HS.values, cbr_hs_preds) if (abs(float(x)-float(y)) < 0.5)) / len(Y_test_HS)
accuracy

0.7989873417721519

In [ ]:
cbr_hs_spreds = hs_cbr.predict(X_stest) 
accuracy = sum(1 for x,y in zip(Y_stest_HS.values, cbr_hs_spreds) if (abs(float(x)-float(y)) < 0.5)) / len(Y_stest_HS)
accuracy

0.6483037974683544

In [ ]:
train_dataset = Pool(data=X_train,
                     label=Y_train_HS)
eval_dataset = Pool(data=X_test,
                     label=Y_test_HS)

# Initialize CatBoostClassifier
hs_cbc = CatBoostClassifier(iterations=7,
                           learning_rate=1,
                           depth=2)

hs_cbc.fit(train_dataset, eval_set=eval_dataset)

0:	learn: 0.5935304	test: 0.4704471	best: 0.4704471 (0)	total: 18.2ms	remaining: 109ms
1:	learn: 0.5842539	test: 0.4544171	best: 0.4544171 (1)	total: 32.3ms	remaining: 80.8ms
2:	learn: 0.5792841	test: 0.4470478	best: 0.4470478 (2)	total: 35.4ms	remaining: 47.2ms
3:	learn: 0.5731603	test: 0.4311998	best: 0.4311998 (3)	total: 38.7ms	remaining: 29ms
4:	learn: 0.5698339	test: 0.4273843	best: 0.4273843 (4)	total: 57.5ms	remaining: 23ms
5:	learn: 0.5657921	test: 0.4227437	best: 0.4227437 (5)	total: 68.3ms	remaining: 11.4ms
6:	learn: 0.5610857	test: 0.4196622	best: 0.4196622 (6)	total: 71.3ms	remaining: 0us

bestTest = 0.419662231
bestIteration = 6



In [ ]:
cbc_hs_preds = hs_cbc.predict(X_test) 
accuracy = sum(1 for x,y in zip(Y_test_HS.values, cbc_hs_preds) if (abs(float(x)-float(y)) < 0.5)) / len(Y_test_HS)
accuracy

0.7954430379746835

In [ ]:
cbc_hs_spreds = hs_cbc.predict(X_stest) 
accuracy = sum(1 for x,y in zip(Y_stest_HS.values, cbc_hs_spreds) if (abs(float(x)-float(y)) < 0.5)) / len(Y_stest_HS)
accuracy

0.5782784810126582

## Abusive

In [ ]:
# Initialize CatBoostRegressor
a_cbr = CatBoostRegressor(iterations=10,
                          learning_rate=1,
                          depth=2)
# Fit model
a_cbr.fit(X_train, Y_train_A)

0:	learn: 0.4335167	total: 2.36ms	remaining: 21.3ms
1:	learn: 0.4263865	total: 6.34ms	remaining: 25.4ms
2:	learn: 0.4208317	total: 10.3ms	remaining: 24.1ms
3:	learn: 0.4191040	total: 14.2ms	remaining: 21.2ms
4:	learn: 0.4170153	total: 17.9ms	remaining: 17.9ms
5:	learn: 0.4120651	total: 35.5ms	remaining: 23.6ms
6:	learn: 0.4104731	total: 38.1ms	remaining: 16.3ms
7:	learn: 0.4090496	total: 42.1ms	remaining: 10.5ms
8:	learn: 0.4076763	total: 58.3ms	remaining: 6.48ms
9:	learn: 0.4062155	total: 62.9ms	remaining: 0us


In [ ]:
cbr_a_preds = a_cbr.predict(X_test) 
accuracy = sum(1 for x,y in zip(Y_test_A.values, cbr_a_preds) if (abs(float(x)-float(y)) < 0.5)) / len(Y_test_A)
accuracy

0.8410126582278481

In [ ]:
cbr_a_spreds = a_cbr.predict(X_stest) 
accuracy = sum(1 for x,y in zip(Y_stest_A.values, cbr_a_spreds) if (abs(float(x)-float(y)) < 0.5)) / len(Y_stest_A)
accuracy

0.8159493670886075

In [ ]:
train_dataset = Pool(data=X_train,
                     label=Y_train_A)
eval_dataset = Pool(data=X_test,
                     label=Y_test_A)

# Initialize CatBoostClassifier
a_cbc = CatBoostClassifier(iterations=3,
                           learning_rate=1,
                           depth=2)

a_cbc.fit(train_dataset, eval_set=eval_dataset)

0:	learn: 0.5518291	test: 0.4105902	best: 0.4105902 (0)	total: 8.44ms	remaining: 16.9ms
1:	learn: 0.5370392	test: 0.3897726	best: 0.3897726 (1)	total: 16.4ms	remaining: 8.2ms
2:	learn: 0.5247188	test: 0.3741719	best: 0.3741719 (2)	total: 22.2ms	remaining: 0us

bestTest = 0.3741718833
bestIteration = 2



In [ ]:
cbc_a_preds = a_cbc.predict(X_test) 
accuracy = sum(1 for x,y in zip(Y_test_A.values, cbc_a_preds) if (abs(float(x)-float(y)) < 0.5)) / len(Y_test_A)
accuracy

0.9

In [ ]:
cbc_a_spreds = a_cbc.predict(X_stest) 
accuracy = sum(1 for x,y in zip(Y_stest_A.values, cbc_a_spreds) if (abs(float(x)-float(y)) < 0.5)) / len(Y_stest_A)
accuracy

0.8139240506329114

# SVM

## Hate Speech

## Abusive

# Random Forest

## Hate Speech

## Abusive

# MultiLayerPerceptron

## Hate Speech

## Abusive